In [1]:
import requests
import json
import time
import os

github_token = "YOUR_ACCESS_TOKEN"

In [2]:
class GitHubDiscussion:
    def __init__(self, token, owner, repo):
        self.token = token
        self.owner = owner
        self.repo = repo
        self.url = "https://api.github.com/graphql"
        self.headers = {"Authorization": f"bearer {self.token}"}

    def run_graphql_query(self, query):
        data = {"query": query}
        response = requests.post(self.url, headers=self.headers, json=data)

        if response.status_code != 200:
            raise Exception(f"Error running GraphQL query: {response.status_code}")

        return response.json()

    def get_discussion_and_comments(self, discussion_number):
        query = f"""
        {{
          repository(owner: "{self.owner}", name: "{self.repo}") {{
            discussion(number: {discussion_number}) {{
              title
              body
              comments(first: 100) {{
                nodes {{
                  author {{
                    login
                  }}
                  body
                  replies(first: 100) {{
                    nodes {{
                      author {{
                        login
                      }}
                      body
                    }}
                  }}
                }}
              }}
            }}
          }}
        }}
        """

        response_data = self.run_graphql_query(query)
        return response_data["data"]["repository"]["discussion"]

    def format_discussion_as_markdown(self, discussion_data, discussion_number):
        discussion_url = f"https://github.com/{self.owner}/{self.repo}/discussions/{discussion_number}"
        markdown_output = f"[URL]( {discussion_url})\n\n# {discussion_data['title']}\n\n{discussion_data['body']}\n\n---\n\n"

        for i, comment in enumerate(discussion_data["comments"]["nodes"], 1):
            markdown_output += f"## Comment {i} by {comment['author']['login']}\n\n{comment['body']}\n\n"

            for j, reply in enumerate(comment["replies"]["nodes"], 1):
                markdown_output += f"### Reply {j} by {reply['author']['login']}\n\n{reply['body']}\n\n"

        return markdown_output

    def save_markdown_output(self, discussion_number):
        discussion_data = self.get_discussion_and_comments(discussion_number)
        markdown_output = self.format_discussion_as_markdown(discussion_data, discussion_number)

        directory = "./../docs/md/"
        if not os.path.exists(directory):
            os.makedirs(directory)

        filename = f"{directory}discussion_{discussion_number}_output.md"

        with open(filename, "w", encoding="utf-8") as file:
            file.write(markdown_output)